In [2]:
import os
#-----------------------------  open import ---------------------------------------
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage import measure,color,io,feature
from scipy.ndimage.morphology import binary_fill_holes

import matplotlib.patches as mpatches

from skimage.morphology import convex_hull_image

from skimage.measure import label, regionprops

import math

from scipy import ndimage as ndi
from skimage import feature

from skimage.filters import threshold_mean

from copy import copy,deepcopy
%matplotlib inline

In [5]:
    path='C:/Users/BEAMCONAN/project_finger/Dataset'
    for fname in os.listdir(path):
        print(fname,end=' ')
        paths=path+'/'+str(fname)
        for iname in os.listdir(paths):
            #print(iname,end=' ')
            pathss=paths+'/'+str(iname)
            img=imread(pathss)
            
            #convert rgb to hsv
            img_hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
        
            #select green only
            img_green=deepcopy(img_hsv)
            img_green[:,:,0]=0
            img_green[:,:,2]=0
            
            #convert hsv green only to grayscale
            img_gray0=img_green[:,:,1]
        
            #thresholding
            ret,thresh = cv2.threshold(img_gray0,50,255,cv2.THRESH_BINARY)
             
            #morprology (dilation)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3)) 
            dilation = cv2.dilate(thresh,kernel,iterations = 1)
            dilation[99:,:]=1
            dilation[30:60,99:]=1
            
            #fill holes
            filled_I = binary_fill_holes(dilation)
            filled_I[99:,:]=0  
            filled_I[30:60,99:]=0
            
            #------------------------------------------------------------
            #feather
            feather_anale=[]
            feather_numberOfFingerTip=[]
            feather_hull=[]
            feather_ratio_area_hull=[]
            #------------------------------------------------------------
            filled_I = filled_I*1
            
            #find contour
            im2,contours,h= cv2.findContours(filled_I,2,1)
            cnt = contours[0]
            
            #find contour area
            contourArea=cv2.contourArea(cnt)
            
            #find convex hull
            hull = cv2.convexHull(cnt,returnPoints = False)
            
            
            hullArea=cv2.contourArea(hull)
            print(hull)
            
            
            defects = cv2.convexityDefects(cnt,hull)
            
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                cv2.line(img,start,end,[153,204,255],2) #0,255,0
                cv2.circle(img,far,1,[0,0,255],-1)
            #********************************************************************
            
            #find center
            M = cv2.moments(cnt)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = (cx,cy)
            cv2.circle(img,center,1,[255,0,0],-1)
            # cv2.Line(img,(cx,cy),1,[255,0,0],-1)
            count_defects = 0
            lena = []
            lenfinger = []
            maxi = 0
            check = 0
            prelen=0
            #find defect
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])

                # find length of all sides of triangle
                #start to end
                a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                #start to defect
                b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                #defect to end
                c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

                # apply cosine rule here
                angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
                
                #center to start check 1 
                longlen = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
                if longlen > maxi:
                    maxi = longlen
                    tip = start

                # ignore angles > 90 and highlight rest with red dots
                if angle <= 92:
                    count_defects += 1
                    cv2.circle(img, far, 1, [255,0,127], -1)
                    #dist = cv2.pointPolygonTest(cnt,far,True)

                    # draw a line from start to end i.e. the convex points (finger tips)
                    # (can skip this part)

                    cv2.line(img,start, end, [255,153,153], 2)
                    cv2.circle(img, start, 1, [153,51,255], -1)
                    cv2.circle(img, end, 1, [153,51,255], -1)
                    lena.append(a)
                    longlen1 = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
                    longlen2 = math.sqrt((end[0] - cx)**2 + (end[1] - cy)**2)
                    if count_defects == 1:
                        lenfinger.append(longlen1)
                        lenfinger.append(longlen2)
                        cv2.line(img,center,start,[255,0,0],1)
                        cv2.line(img,center,end,[255,0,0],1)
                    else:
                        lenfinger.append(longlen2)
                        cv2.line(img,center,end,[255,0,0],1)
            #********************************************************************
            
            
            #********************************************************************
            t="11";
            # define actions required
            if count_defects == 1:
                t="222222222"
            elif count_defects == 2:
                if lena[1]>lena[0]:
                    if lena[1] > 45 :
                        t="333333333"  
                    elif lena[1] >28 :
                        t="888888888"
                    else:
                        t="999999999"

                else:
                    if lena[0] > 28 :
                        t="777777777"
                    else:
                        t="999999999"  

                #separate 3 6 7 8 9
            elif count_defects == 3:
                t="444444444"
            elif count_defects == 4:
                t="555555555"
            else:
                if tip[1]<cy:
                    cv2.circle(img, tip, 1, [153,51,255], -1)
                    cv2.line(img,center,tip,[255,0,0],1)
                    t="11111111"
                else:
                    t="00000000"
            #*******************************************************************
            fig,ax =plt.subplots(figsize=(5,5))
            plt.title(t)
            ax.imshow(img)

0 

error: OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:272: error: (-215:Assertion failed) npoints >= 0 && (depth == 5 || depth == 4) in function 'cv::contourArea'


In [115]:
    path='C:/Users/BEAMCONAN/project_finger/Dataset'
    for fname in os.listdir(path):
        print(fname,end=' ')
        paths=path+'/'+str(fname)
        for iname in os.listdir(paths):
            #print(iname,end=' ')
            pathss=paths+'/'+str(iname)
            img=imread(pathss)
            
            img_hsv=color.convert_colorspace(img,'RGB','HSV')
           
        
            #imshow(green)
            img_hsv[:,:,0]=0
            img_hsv[:,:,2]=0
            
            img_rgb=img_hsv[:,:,1]
            #img_gray=color.rgb2gray(img_rgb)
            
            
            
            img_gray = cv2.cvtColor(img_rgb,cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(img_gray,100,200)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2)) 
            dilation = cv2.dilate(edges,kernel,iterations = 1)
            dilation[99:,:]=1
            dilation[30:60,99:]=1
            
            
            binary=np.logical_not((dilation<0.2))
            filled_I = binary_fill_holes(binary)
            filled_I[99:,:]=0  
            filled_I[30:60,99:]=0
            
            #img_gray1 = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            #edges1 = cv2.Canny(img_gray1,100,200)
            #dilation1 = cv2.dilate(edges1,kernel,iterations = 1)
            #dilation1[99:,:]=1
            #dilation1[30:60,99:]=1
            
            #filled_I1 = binary_fill_holes(dilation)
            #filled_I1[99:,:]=0  
            #filled_I1[30:60,99:]=0
            
            #img_b=cv2.bitwise_or(filled_I,filled_I1)
            
            
            hull = convex_hull_image(filled_I)
            contours = measure.find_contours(filled_I, 0.8)
            cont=measure.find_contours(hull,0.8)
            
            # Display the image and plot all contours found
            fig, ax = plt.subplots()
            ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)

            for n, contour in enumerate(contours):
                ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

            for n, contour in enumerate(cont):
                ax.plot(contour[:, 1], contour[:, 0], linewidth=2)    

0 

error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'
